The point of this project is to train an AI agent on all the content I've written for my personal blog so I can have a brand-informed content creator assistant.

In [1]:
!pip install faiss-cpu
!pip install openai
!pip install sentence-transformers
!pip install langchain
!pip install langchain_openai
!pip install --quiet langchain_experimental langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 

setup openai api key

In [2]:
import os

os.environ['OPENAI_API_KEY'] = 'get-your-own-key!'

read all of my blog posts as one long string, then chunk them into semantic docs

In [4]:
# This is a long document we can split up.
with open('/content/my_blog_posts.txt') as f:
    blog_posts = f.read()

# Create Text Splitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(OpenAIEmbeddings())

# Split Text into docs
blog_post_docs = text_splitter.create_documents([blog_posts])

In [5]:
# Use this to test
print(blog_post_docs[0].page_content)

﻿Beginners Guide to Productivity (5 Tips to Getting Things Done)
Jack Ross
Jack Ross


9 min read
·
Feb 16, 2024


Are you tired of watching the clock tick by, feeling like you haven’t accomplished anything? Do you end the day wondering where all your time went and why you still have so many things left on your to-do list? For years I was caught in this same cycle of laziness and procrastination, watching my dreams and daily goals slip away, one unproductive day after another…


But over the last 5 years of studying time management and productivity, I’ve transformed myself from someone who used to procrastinate until the last second into someone who takes action by default. And with just a few changes to your routine, you could transform your days from unproductive to unbelievably efficient. Whether you’re a student, a professional, or just someone looking to get more out of your day, these 9 tips are designed to help you maximize your productivity. These are the mindsets, tools, and m

split docs, create embeddings, and save to FAISS db

> Add blockquote



In [6]:
# Create Text Splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(blog_post_docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
db = FAISS.from_documents(splits, embeddings)

save embeddings

In [7]:
db.save_local("blog_post_embeddings")

We now have embeddings for each 'doc' in our set of blog posts.

In [8]:
# load the embeddings

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Load the local FAISS index
store = FAISS.load_local(
    folder_path='blog_post_embeddings',
    embeddings=OpenAIEmbeddings(model="text-embedding-3-large"),
    allow_dangerous_deserialization=True
)
retriever = store.as_retriever()

In [40]:
# Query FAISS index for relevant chunks related to productivity
query = "productivity and time management"
config = {'query': query, 'k': 5}  # Adjust k as needed
retrieved_chunks = await retriever.ainvoke('aget_relevant_documents', config=config)

# Concatenate retrieved chunks into a single text
productivity_article = "\n".join([chunk.content for chunk in retrieved_chunks if hasattr(chunk, 'content')])

# Generate article using AI assistant model with chat completion
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": productivity_article},
        {"role": "user", "content": "You are an AI content creator assistant for Jack Ross and you are trained on articles he has written. Generate an article in Jacks style about productivity and time management"},
    ],
    model="gpt-4-turbo",
)

generated_article = response.choices[0].message.content.strip()
print(generated_article)

## Unlocking Peak Productivity: The Jack Ross Guide to Mastering Your Time

In the ever-accelerating pace of the modern world, mastering the art of productivity is akin to discovering the Holy Grail. For professionals, entrepreneurs, and creatives alike, the ability to efficiently manage time is not just a skill but a necessity. Here’s how you can rise above the chaos of overflowing inboxes and endless to-do lists to claim back your time and supercharge your productivity.

### 1. **Embrace Strategic Planning**

"Fail to plan, plan to fail." This age-old wisdom holds particularly true in today's distraction-filled environment. Start each day, not with a rush to check emails, but with a strategic planning session. Spend 10-15 minutes each morning mapping out your day. Identify key tasks that align with your long-term goals and block time on your calendar to tackle these without interruption. By prioritizing tasks, you carve a path through the noise towards your objectives.

### 2. **The 